<a href="https://colab.research.google.com/github/dpcks/DACON--AI-/blob/main/bestmodel(cat123)_(%EC%9B%90%EB%B3%B8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import

In [ ]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer

import torch

from tqdm.auto import tqdm
from sklearn.metrics import f1_score

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

# mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_PATH = "/content/drive/MyDrive/딥러닝 프로젝트/data//"
SEED = 88

In [ ]:
if not os.path.isdir("image"):
    !cp "{DATA_PATH}open.zip" "open.zip"
    !unzip -qq "open.zip"

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train

,id,img_path,overview,cat1,cat2,cat3
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식
...,...,...,...,...,...,...
16981,TRAIN_16981,./image/train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,음식,음식점,한식
16982,TRAIN_16982,./image/train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,숙박,숙박시설,모텔
16983,TRAIN_16983,./image/train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,숙박,숙박시설,모텔
16984,TRAIN_16984,./image/train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,레포츠,육상 레포츠,"야영장,오토캠핑장"


In [ ]:
test

,id,img_path,overview
0,TEST_00000,./image/test/TEST_00000.jpg,신선한 재료로 정성을 다해 만들었다. 늘 변함없는 맛과 서비스로 모실것을 약속한다.
1,TEST_00001,./image/test/TEST_00001.jpg,"청청한 해역 등량만과 율포해수욕장이 한눈에 내려다 보이는 위치에 있으며, 막 잡은 ..."
2,TEST_00002,./image/test/TEST_00002.jpg,장터설렁탕은 남녀노소 누구나 즐길 수 있는 전통 건강식으로 좋은 재료와 전통 조리방...
3,TEST_00003,./image/test/TEST_00003.jpg,다양한 형태의 청소년수련활동을 제공함으로써 청소년들이 민주사회의 주역이 될 수 있도...
4,TEST_00004,./image/test/TEST_00004.jpg,팔공산은 경산시의 북쪽에 위치한 해발 1192.3 m의 높은 산으로 신라시대에는 중...
...,...,...,...
7275,TEST_07275,./image/test/TEST_07275.jpg,"막국수와 수육을 주메뉴로 하며, 넓은 주차장이 마련되어 있어 주차하기 편리하다.<br>"
7276,TEST_07276,./image/test/TEST_07276.jpg,"통진두레문화센터는 우리고유의 전통무형문화와 민속예술을 계승/발전 시키고, 다양한 문..."
7277,TEST_07277,./image/test/TEST_07277.jpg,"수도권에서 가까운 위치, 문산천을 따라 걷는 산책코스, 한여름 더위를 날려버릴 시원..."
7278,TEST_07278,./image/test/TEST_07278.jpg,\n전남 구례군 관산리에 위치한 노고단 게스트하우스&호텔은 지리산을 파노라마로 관망...


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16986 entries, 0 to 16985
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        16986 non-null  object
 1   img_path  16986 non-null  object
 2   overview  16986 non-null  object
 3   cat1      16986 non-null  object
 4   cat2      16986 non-null  object
 5   cat3      16986 non-null  object
dtypes: object(6)
memory usage: 796.3+ KB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7280 entries, 0 to 7279
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7280 non-null   object
 1   img_path  7280 non-null   object
 2   overview  7280 non-null   object
dtypes: object(3)
memory usage: 170.8+ KB


In [ ]:
train.nunique()

id          16986
img_path    16986
overview    16973
cat1            6
cat2           18
cat3          128
dtype: int64

In [ ]:
test.nunique()

id          7280
img_path    7280
overview    7280
dtype: int64

In [ ]:
le = LabelEncoder()
train["cat1"] = le.fit_transform(train["cat1"].to_numpy())
train["cat2"] = le.fit_transform(train["cat2"].to_numpy())
train["target"] = le.fit_transform(train['cat3'].to_numpy())
train

,id,img_path,overview,cat1,cat2,cat3,target
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,5,13,항구/포구,120
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,0,11,골프,8
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,3,12,한식,118
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,3,12,한식,118
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,3,12,한식,118
...,...,...,...,...,...,...,...
16981,TRAIN_16981,./image/train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,3,12,한식,118
16982,TRAIN_16982,./image/train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,2,9,모텔,31
16983,TRAIN_16983,./image/train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,2,9,모텔,31
16984,TRAIN_16984,./image/train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,0,11,"야영장,오토캠핑장",73


In [ ]:
!pip install kiwipiepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 34.8 MB/s 
     |████████████████████████████████| 30.5 MB 1.3 MB/s 
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.14.0-py3-none-any.whl size=30560039 sha256=c0bc9cc9583f14691788fcf5e24d3516c84049622a69d85aa46558ec35d3371d
  Stored in directory: /root/.cache/pip/wheels/97/36/50/1fc99e4a3ce636ed30739cc2cb333b36e6e3778f365f1adf77
Successfully built kiwipiepy-model


In [ ]:
from kiwipiepy import Kiwi

kiwi = Kiwi()
kiwi.prepare()

In [ ]:
lst = train["overview"].tolist()
gen = kiwi.analyze(lst)
train_overview = []

for tokens in tqdm(gen,total=len(lst)):
    token = [ token.form for token in tokens[0][0] if token.tag[0] in "NNP" or "IC"]
    train_overview.append(" ".join(token))

lst = test["overview"].tolist()
gen = kiwi.analyze(lst)
test_overview = []
for tokens in tqdm(gen,total=len(lst)):
    token = [ token.form for token in tokens[0][0] if token.tag[0] in "NNP" or "IC" ]
    test_overview.append(" ".join(token))

  0%|          | 0/16986 [00:00<?, ?it/s]

  0%|          | 0/7280 [00:00<?, ?it/s]

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 31.6 MB/s 
     |████████████████████████████████| 163 kB 74.0 MB/s 
     |████████████████████████████████| 7.6 MB 64.0 MB/s 


In [ ]:
model_name = "klue/bert-base" #허깅페이스

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
train_overview = np.array(train_overview)
test_overview = np.array(test_overview)
cat1 = np.array(train["cat1"])
cat2 = np.array(train["cat2"])
target = train.target.to_numpy()

In [ ]:
token = tokenizer(train_overview[0], add_special_tokens=True,padding="max_length", truncation=True)
token

{'input_ids': [2, 1282, 2283, 2421, 1497, 5574, 1889, 1, 1264, 3603, 4973, 7960, 1504, 8808, 13171, 3603, 12168, 648, 6277, 1498, 1889, 3709, 772, 1497, 4429, 1498, 4705, 594, 1513, 3745, 4822, 10290, 2115, 991, 848, 4455, 1889, 809, 18, 1896, 4240, 1421, 4198, 859, 1, 25014, 882, 1497, 1264, 11336, 882, 1503, 11807, 1889, 1, 29223, 1498, 8318, 944, 2468, 818, 594, 1513, 3745, 19723, 904, 1264, 1503, 13104, 1497, 4822, 1503, 7524, 1500, 1498, 4311, 1873, 594, 1513, 793, 885, 1889, 809, 18, 3819, 16, 1264, 1421, 793, 544, 1325, 1428, 7769, 16, 9191, 7076, 886, 1503, 11016, 543, 5214, 1406, 9005, 594, 1513, 3745, 16, 5088, 20584, 1875, 2051, 2021, 12900, 1504, 5166, 594, 1513, 809, 18, 32, 69, 2008, 34, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self ,tokenizer , x, cat1 = None, cat2 = None,  y = None ): 
        self.tokenizer = tokenizer
        self.x = x
        self.cat1 = cat1
        self.cat2 = cat2
        self.y = y
    def __len__(self): 
        return self.x.shape[0]
    def __getitem__(self, idx): 
        item = {}
        item["x"] = self.__tokenizer(self.x[idx]) 
        if self.cat1 is not None:
            item["cat1"] = torch.tensor(self.cat1[idx])
        if self.cat2 is not None:
            item["cat2"] = torch.tensor(self.cat2[idx])
        if self.y is not None:
            item["y"] = torch.tensor(self.y[idx])
        return item
    def __tokenizer(self,text):
        inputs = self.tokenizer(text, add_special_tokens=True,padding="max_length", truncation=True)
        for k, v in inputs.items(): 
            inputs[k] = torch.LongTensor(v) 
        return inputs

In [ ]:
dt = MyDataset(tokenizer,train_overview,cat1,cat2,target)
dl = torch.utils.data.DataLoader(dt, batch_size=2,shuffle=False) 
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[    2,  1282,  2283,  ...,     0,     0,     0],
         [    2,  4619, 16781,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])},
 'cat1': tensor([5, 0]),
 'cat2': tensor([13, 11]),
 'y': tensor([120,   8])}

In [ ]:
model = AutoModel.from_pretrained(model_name)

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
batch["x"].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
batch

{'x': {'input_ids': tensor([[    2,  1282,  2283,  ...,     0,     0,     0],
         [    2,  4619, 16781,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])},
 'cat1': tensor([5, 0]),
 'cat2': tensor([13, 11]),
 'y': tensor([120,   8])}

In [ ]:
outputs = model(**batch["x"])
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [ ]:
model.config.hidden_size

768

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, model_name): 
        super().__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.output_layer1 = torch.nn.Linear(self.model.config.hidden_size,6)
        self.output_layer2 = torch.nn.Linear(self.model.config.hidden_size,18)
        self.output_layer3 = torch.nn.Linear(self.model.config.hidden_size, 128)
    def forward(self, x):
        x = self.model(**x)
        out1 = self.output_layer1(x[0][:,0])
        out2 = self.output_layer2(x[0][:,0]) 
        out3 = self.output_layer3(x[0][:,0])
        return out1, out2, out3

In [ ]:
model = Net(model_name)
model(batch["x"])

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(tensor([[-0.5672,  0.2706, -0.2813, -0.2443,  0.7579,  0.4198],
         [-0.8187,  0.7218,  0.5524, -1.1336, -0.0846,  0.2989]],
        grad_fn=<AddmmBackward0>),
 tensor([[ 0.1319,  0.2819, -0.3481,  0.2654,  0.5345,  0.6520, -0.6699, -0.7110,
           0.3017,  0.2337, -0.2985, -0.2426,  0.8566,  0.2414,  0.2680, -1.0036,
           0.3128,  0.0149],
         [ 0.0746,  0.1743, -0.3791,  0.4343, -0.2388,  0.5311,  0.1864, -0.3212,
           0.0643, -0.3103, -0.7951, -0.4259,  0.9582,  0.1127, -0.1240, -0.6395,
          -0.0714, -0.2040]], grad_fn=<AddmmBackward0>),
 tensor([[ 1.0447,  0.6303, -0.2319, -0.1796,  0.4406,  0.2425, -0.1566, -0.4578,
          -0.0470,  0.0292,  0.3088, -0.5476, -0.2395,  0.7763, -0.4282, -0.4528,
           0.6328, -0.5933,  0.0521, -0.1705,  0.9295,  0.9198,  0.8289,  0.6663,
           0.2648, -0.9424,  0.0960, -0.1635, -0.7320, -0.2138,  0.1180,  0.2755,
          -0.7111,  0.5391, -0.4666, -0.0501, -0.7664, -1.3784, -0.7824, -1.0050,
          

In [ ]:
train.target.nunique()

128

In [ ]:
def train_loop(dataloader,model,loss_fn,optimizer,device):
    epoch_loss = 0 
    model.train() 
    for batch in tqdm(dataloader): 
        pred1,pred2,pred3 = model(batch["x"].to(device)) 
        loss1 = loss_fn(pred1, batch["cat1"].to(device))
        loss2 = loss_fn(pred2, batch["cat2"].to(device))
        loss3 = loss_fn(pred3, batch["y"].to(device)) 

        loss = loss1 * 0.05 + loss2 * 0.1 + loss3 * 0.85
        
        optimizer.zero_grad() 
        loss.backward()  
        optimizer.step() 
        
        epoch_loss += loss.item() 

    epoch_loss /= len(dataloader) 

    return epoch_loss 

In [ ]:
@torch.no_grad() 
def test_loop(dataloader,model,loss_fn,device): 
    epoch_loss = 0
    model.eval() 

    
    pred_list = []
    softmax = torch.nn.Softmax(dim=1) 

    for batch in tqdm(dataloader):
        
        _,_,pred3 = model(batch["x"].to(device))
        if batch.get("y") is not None: 
            loss = loss_fn(pred3, batch["y"].to(device))
            epoch_loss += loss.item()
        
        pred = softmax(pred3)
        pred = pred.to("cpu").numpy() 
        pred_list.append(pred)

    epoch_loss /= len(dataloader)

    pred = np.concatenate(pred_list) 
    return epoch_loss , pred 

In [ ]:
n_splits = 5
batch_size = 4
epochs = 100
loss_fn = torch.nn.CrossEntropyLoss() # 다중분류 손실객체

In [ ]:
DATA_PATH2 = "/content/drive/MyDrive/딥러닝 프로젝트/model_weight/"

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
cv = KFold(n_splits=n_splits,shuffle=True, random_state=SEED)

In [ ]:
is_holdout = False
seed_everything(SEED)
best_score_list = []
for i,(tri,vai) in enumerate(cv.split(train_overview)):
    if i <= 1:
        model = Net(model_name).to(device)
        optimizer = torch.optim.Adam(model.parameters(),lr=0.00001)

        train_dt = MyDataset(tokenizer,train_overview[tri],cat1[tri],cat2[tri],target[tri])
        valid_dt = MyDataset(tokenizer,train_overview[vai],cat1[vai],cat2[vai],target[vai])
        train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)
        valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size=batch_size,shuffle=False)

        best_score = 0
        patience = 0

        for epoch in range(epochs):
            
            train_loss = train_loop(train_dl, model, loss_fn,optimizer,device )
            valid_loss , pred = test_loop(valid_dl, model, loss_fn,device  )
            
            pred = np.argmax(pred, axis=1) 
            true = target[vai] 
            score = f1_score(true, pred , average="weighted")
            print(train_loss,valid_loss,score)
            patience += 1
            if best_score < score:
                patience = 0
                best_score = score
                torch.save(model.state_dict(),f"{DATA_PATH2}premodel_{i}.pth")

            if patience == 2:
                break
        print(f"Fold ({i}), BEST F1: {best_score}")
        best_score_list.append(best_score)

        torch.cuda.empty_cache()

        if is_holdout:
            break

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/3397 [00:00<?, ?it/s]

  0%|          | 0/850 [00:00<?, ?it/s]

1.282631786448581 0.838810437120059 0.7706722021773744


  0%|          | 0/3397 [00:00<?, ?it/s]

...

1.3034683637251927 0.811805838619304 0.7793885218136429


  0%|          | 0/3398 [00:00<?, ?it/s]

  0%|          | 0/850 [00:00<?, ?it/s]

0.6104883082197862 0.7026458620282766 0.8113120783645926


  0%|          | 0/3398 [00:00<?, ?it/s]

In [ ]:
np.mean(best_score_list)

In [ ]:
test_dt = MyDataset(tokenizer,test_overview)
test_dl = torch.utils.data.DataLoader(test_dt, batch_size=batch_size, shuffle=False)
pred_list = []
for i in range(n_splits):
    model = Net(model_name).to(device)
    state_dict = torch.load(f'{DATA_PATH2}premodel_{i}.pth')
    model.load_state_dict(state_dict)
    _ , pred = test_loop(test_dl, model, loss_fn,device  )
    pred_list.append(pred)
    if is_holdout:
        break

In [ ]:
pred = np.mean(pred_list,axis=0) 
pred = np.argmax(pred, axis=1) 
pred

In [ ]:
submit = pd.read_csv("sample_submission.csv")
submit["cat3"] = le.inverse_transform(pred)
submit

In [ ]:
DATA_PATH3 = "/content/drive/MyDrive/딥러닝 프로젝트/summit/"

In [ ]:
submit.to_csv(f'{DATA_PATH3}pre_submit.csv', index=False)